# Convert .laz to .las

In [1]:
import numpy as np

# Read the PCD file into a NumPy array
data = np.fromfile("/forest_3d_app/3d_forest/forest_3d_app/data/pcd_files/060_2-295_2_slam.pcd", dtype=np.float32)

# Print the number of points in the PCD file
print(data.shape[0])

# Print the coordinates of the first point in the PCD file
print(data[0])
print(data.shape)

57160247
11685367000.0
(57160247,)


In [2]:

import os, sys
sys.path.append("../../")
from src.forest3D.pcd import read_pcd


path = "/forest_3d_app/3d_forest/forest_3d_app/data/pcd_files/150_2-160_2_slam.pcd"
# Import the Pandas dataframe.
pcd_file = read_pcd(path)
xyz_data_pcd = pcd_file['points'][['x','y','z']].values


In [3]:
import sys
sys.path.insert(0, '../')
from forest3D import lidar_IO,ground_removal,treeDetector as treeDetector,inventory_tools,utilities

import os
import json
import numpy as np

output_dir = '../../data/'
grid_size = [600, 600, 1000]
grid_res = 0.1
# print("reading the data into python...")
# #path = '/home/lloyd/Documents/datasets/lidar/forestry_usyd/extracts/plots_for_training/V1_Scanner1_161011_220153_crop026.asc'
# #xyz_data = lidar_IO.XYZreadFromCSV(path,delimiter=' ',x=0,y=1,z=2)
# #path = '/home/lloyd/Documents/datasets/lidar/forestry_usyd/FCNSW_FertTrialAOI_VUX1_conv.las'
# path = '../../data/Forest.las'
# xyz_data = lidar_IO.readFromLas(path, fields = ['x','y','z'])

path = "/forest_3d_app/3d_forest/forest_3d_app/data/pcd_files/060_2-295_2_slam.pcd"
# Import the Pandas dataframe.
pcd_file = read_pcd(path)
xyz_data = pcd_file['points'][['x','y','z']].values


print("removing the ground...")
offset = [0,0,0]       # optional: use to make calculations stable on large values
xyz_data_gr = ground_removal.removeGround(xyz_data,offset,thresh=2.0,proc_path=output_dir)
ground_pts = ground_removal.load_ground_surface(os.path.join(output_dir,'_ground_surface.ply'))



config_dict = {
    "doHisteq": [
        False,
        True
    ],
    "raster_layers": [
        "vertical_density",
        "canopy_height"
    ],
    "gridSize": [
        600,
        600,
        1000
    ],
    "res": 0.1,
    "support_window": [
        11,
        7,
        1
    ],
    "normalisation": "rescale+histeq"
}
rasterTreeDetector = treeDetector.RasterDetector(**config_dict )


TabError: inconsistent use of tabs and spaces in indentation (object_detectors.py, line 169)

In [ ]:
import sys
sys.path.insert(0, '../')
from forest3D import lidar_IO,ground_removal,treeDetectorYolov3,inventory_tools,utilities
detector_addr = "/forest_3d_app/3d_forest/forest_3d_app/models/models/detection/hvp/"
labels = rasterTreeDetector.sliding_window(detector_addr,xyz_data_gr,ground_pts=ground_pts,windowSize = [600,600],stepSize = 80)



print("outputing detection results...")
lidar_IO.writePly_labelled(os.path.join(output_dir,'detection.ply'),xyz_data_gr,labels,offset)




In [ ]:
print("outputting inventory...")
tree_tops = inventory_tools.get_tree_tops(xyz_data_gr,labels)
heights = inventory_tools.get_tree_heights(xyz_data_gr[:,:3],ground_pts)
inventory = np.hstack((xyz_data_gr,heights[:,np.newaxis]))
utilities.write_csv(os.path.join(output_dir,'inventory.csv'),inventory,header='x,y,z,id,height')